## En este notebook queda registrado el código mediante el que fueron creadas las tablas de la base de datos de mysql alojada en AWS

#### Definimos las funciones mediantes las que atacar a la base de datos

In [1]:
#Importamos la librerías y variables necesarias
import mysql.connector
import sqlite3
import pandas as pd
import numpy as np
from flask import Flask, jsonify, request
import pickle
import os
#################################################################################
#################################################################################
"""Definimos las variables generales para atacar la base de datos"""
#Definimos un cursor con el que atacar a la DB
cnx = mysql.connector.connect(
    user="admin",
    password="admin123",
    host="test-db.cze2nnbbx5pc.eu-west-3.rds.amazonaws.com",
    database="prueba")
cursor = cnx.cursor()
#Definimos una función mediante la que ejecutar las querys
def make_query(code):
    cursor.execute(code)
    results = cursor.fetchall()
    cnx.commit()
    return(results)
#Definimos una función mediante la que ejecutar las querys devolviendo un dataframe
def make_query_dataframe(code):
    cursor.execute(code)
    results = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]  # Obtener los nombres de las columnas
    df = pd.DataFrame(results, columns=column_names)  # Crear el DataFrame
    return df

#### 1. employee_raw

In [ ]:
#cargamos los datos del CSV
datos_raw=pd.read_csv("hr_names.csv", index_col="EmployeeID")
data=datos_raw.drop(columns=["EmployeeCount","Over18","StandardHours"])
#Cambiamos el nombre de las columnas
order=['Name','hrs', 'JobInvolvement', 'PerformanceRating', 'EnvironmentSatisfaction',
       'JobSatisfaction', 'WorkLifeBalance', 'Age', 'Attrition',
       'BusinessTravel', 'Department', 'DistanceFromHome', 'Education',
       'EducationField', 'Gender', 'JobLevel', 'JobRole', 'MaritalStatus',
       'MonthlyIncome', 'NumCompaniesWorked', 'PercentSalaryHike',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager'
       ]
data = data.reindex(columns=order)
#Creamos la query para la creación de la tabla y definimos los tipos de campos
make_query("""CREATE TABLE employee_raw (
    id_employee INT NOT NULL,
    name VARCHAR(50),
    hours DECIMAL(4,2),
    involvement VARCHAR(50),
    performance VARCHAR(50),
    environment VARCHAR(50),
    satisfaction VARCHAR(50),
    life_balance VARCHAR(50),
    age INT,
    attrition VARCHAR(50),
    travel VARCHAR(50),
    department VARCHAR(50),
    distance_home INT,
    education VARCHAR(50),
    education_field VARCHAR(50),
    gender VARCHAR(50),
    job_level INT,
    role VARCHAR(50),
    marital_status VARCHAR(50),
    income INT,
    previous_companies INT,
    salary_hike INT,
    stock_option INT,
    total_working_years INT,
    training_ly INT,
    years_company INT,
    last_promotion INT,
    years_curr_manager INT,
    PRIMARY KEY (id_employee))"""
)
data=data.reset_index()
# Ejecutamos la query fila por fila para acelerar el proceso
for row in data.itertuples(index=False):
    values = ', '.join("'" + str(value).replace("'", "''") + "'" for value in row)
    query = f"INSERT INTO employee_raw VALUES ({values})"
    cursor.execute(query)
    print(row)
# Confirmar los cambios en la base de datos
cnx.commit()
# Cerrar el cursor y la conexión
cursor.close()
cnx.close()

#### 2. current_employees

In [ ]:
#Hacemos una query para traer a todos los trabajadores raw
datos_base_datos=make_query_dataframe("SELECT * FROM prueba.employee_raw;")
#Seleccionamos a los trabajadores actuales
employess_actuales=datos_base_datos[datos_base_datos["attrition"]=="No"].copy()
employess_actuales=employess_actuales.drop(columns="id_employee").reset_index().drop(columns="index").reset_index().rename(columns={'index': 'id_employee'})

#Creamos la query para la creación de la tabla y definimos los tipos de campos
make_query("""CREATE TABLE current_employees (
    id_employee INT,
    name VARCHAR(50),
    hours DECIMAL(4,2),
    involvement VARCHAR(50),
    performance VARCHAR(50),
    environment VARCHAR(50),
    satisfaction VARCHAR(50),
    life_balance VARCHAR(50),
    age INT,
    attrition VARCHAR(50),
    travel VARCHAR(50),
    department VARCHAR(50),
    distance_home INT,
    education VARCHAR(50),
    education_field VARCHAR(50),
    gender VARCHAR(50),
    job_level INT,
    role VARCHAR(50),
    marital_status VARCHAR(50),
    income INT,
    previous_companies INT,
    salary_hike INT,
    stock_option INT,
    total_working_years INT,
    training_ly INT,
    years_company INT,
    last_promotion INT,
    years_curr_manager INT,
    PRIMARY KEY (id_employee))"""
)

# Cargar los datos del CSV a la tabla
for row in employess_actuales.itertuples(index=False):
    values = ', '.join("'" + str(value).replace("'", "''") + "'" for value in row)
    query = f"INSERT INTO current_employees VALUES ({values})"
    cursor.execute(query)
    print(row)
# Confirmar los cambios en la base de datos
cnx.commit()
# Cerrar el cursor y la conexión
cursor.close()
cnx.close()

#### 3. predictions

In [ ]:
make_query("""CREATE TABLE predictions (
    id_employee INT,
    months_left DECIMAL(8,2),
    PRIMARY KEY (id_employee))"""
)

numeros_aleatorios_prueba=np.random.randint(1,36,3698)
df=pd.DataFrame(numeros_aleatorios_prueba).rename(columns={0:"months_left"}).reset_index()
# Ejecutamos la query fila por fila para acelerar el proceso
for row in df.itertuples(index=False):
    values = ', '.join("'" + str(value).replace("'", "''") + "'" for value in row)
    query = f"INSERT INTO predictions VALUES ({values})"
    cursor.execute(query)
    print(row)
# Confirmar los cambios en la base de datos
cnx.commit()
# Cerrar el cursor y la conexión
cursor.close()
cnx.close()

#### 4. replacement_time

In [ ]:
#Creación de la tabla del tiempo medio necesario para reemplazar a un puesto determinado
data = {
    'JobRole': [
        'Manufacturing Director',
        'Laboratory Technician',
        'Research Director',
        'Sales Representative',
        'Sales Executive',
        'Research Scientist',
        'Manager',
        'Healthcare Representative',
        'Human Resources'
    ],
    'Avg replacement time (monthly)': [8.0, 4.5, 8.0, 4.5, 8.0, 4.5, 8.0, 4.5, 4.5]
}

df = pd.DataFrame(data)
df=df.reset_index()


# Ejecutamos la query fila por fila para acelerar el proceso
for row in df.itertuples(index=False):
    values = ', '.join("'" + str(value).replace("'", "''") + "'" for value in row)
    query = f"INSERT INTO replacement_time VALUES ({values})"
    cursor.execute(query)
    print(row)
# Confirmar los cambios en la base de datos
cnx.commit()
# Cerrar el cursor y la conexión
cursor.close()
cnx.close()

#Procedimiento creado para la actualización tras editar la tabla de predicciones
DELIMITER //

CREATE PROCEDURE actualizacion_predictions()
BEGIN
    -- Crear la tabla prueba_miguel si no existe
    CREATE TABLE IF NOT EXISTS prueba_miguel (
        id_employee INT,
        replacement_cost DECIMAL(10, 2),
        months_left INT,
        risk VARCHAR(10)
    );

    -- Eliminar todos los datos existentes en la tabla prueba_miguel
    DELETE FROM prueba_miguel;

    -- Insertar los nuevos datos actualizados en prueba_miguel
    INSERT INTO prueba_miguel (id_employee, replacement_cost, months_left, risk)
    SELECT 
        c.id_employee,
        c.income * 12 * 0.645 AS replacement_cost,
        p.total_months_in_company - (c.years_company * 12) AS months_left,
        (CASE
            WHEN (percent_rank() OVER (ORDER BY (p.total_months_in_company - (c.years_company * 12))) <= 0.25) THEN 'Very high'
            WHEN (percent_rank() OVER (ORDER BY (p.total_months_in_company - (c.years_company * 12))) <= 0.5) THEN 'High'
            WHEN (percent_rank() OVER (ORDER BY (p.total_months_in_company - (c.years_company * 12))) <= 0.75) THEN 'Medium'
            ELSE 'Low'
        END) AS risk
    FROM current_employees c
    LEFT JOIN replacement_time r ON (c.role = r.role)
    LEFT JOIN predictions p ON (c.id_employee = p.id_employee)
    ORDER BY c.id_employee ASC;
END;
//

DELIMITER ;
